In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = {'png', 'retina'}

In [14]:
import os, zipfile, io, re
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception
from keras.models import Model, load_model
from keras.layers.core import Dense
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from glob import glob

In [5]:
img_size = 200
classes = ['male', 'female']
num_classes = len(classes)

In [8]:
%%time

imgs = glob('data/UTKFace/*.jpg')

X = []
y = []

for img in imgs:
    img_data = Image.open(img)
    np_data = np.asarray(img_data)
    filename = os.path.basename(img)
    filename_split = [i for i in filename.split('_')]
    X.append(np_data)
    y.append(filename_split[1])
    
X = np.array(X)
y = np.array(y)

CPU times: user 16.7 s, sys: 1.54 s, total: 18.2 s
Wall time: 23.3 s


In [9]:
X.shape, y.shape

((23697, 200, 200, 3), (23697,))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)
del X, y
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((21327, 200, 200, 3), (21327,), (2370, 200, 200, 3), (2370,))

In [12]:
X_train = X_train.astype('float32') / 255
x_test = X_test.astype('float32') / 255

y_train = to_categorical(y_train, num_classes = num_classes)
y_test = to_categorical(y_test, num_classes = num_classes)

In [15]:
base_model = Xception(include_top = False,
                     weights = "imagenet",
                     input_shape = None)

83689472/83683744 [==============================] - 14s 0us/step


In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [26]:
datagen = ImageDataGenerator(featurewise_center=False,
                            samplewise_center=False,
                            featurewise_std_normalization=False,
                            samplewise_std_normalization=False,
                            zca_whitening=False,
                            rotation_range=0,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=False)

In [20]:
#EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss',
                              patience=10,
                              verbose=1)

#ModelCheckpoint
weights_dir = 'data/weights/'
if os.path.exists(weights_dir) == False:
    os.mkdir(weights_dir)
    
model_checkpoint = ModelCheckpoint(weights_dir + "val_loss{val_loss:.3f}.hdf5",
                                  monitor='val_loss',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=True,
                                  period=3)

#reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.1,
                             patience=3,
                             verbose=1)

#log for TensorBoard
tb_log = TensorBoard(log_dir = "logs/")

In [23]:
model = Model(inputs=base_model.input, output=predictions)

#freeze weights till 108 layers
for layer in model.layers[:108]:
    layer.trainable = False
    
    #unfreeze BN layers
    if layer.name.startswith('batch_normalization'):
        layer.trainable = True
    if layer.name.endswith('bn'):
        layer.trainable = True
    
for layer in model.layers[108:]:
    layer.trainable = True
    
#After setting trainable to the layers for 108th layer and more,
#compile the model
model.compile(optimizer=Adam(),
             loss='categorical_crossentropy',
             metrics=["accuracy"])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [ ]:
%%time

hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                          steps_per_epoch = X_train.shape[0] // 32,
                          epochs = 50,
                          validation_data=(X_test, y_test),
                          callbacks=[early_stopping, reduce_lr, tb_log],
                          shuffle=True,
                          verbose=1)

Epoch 1/50
666/666 [==============================] - 310s 465ms/step - loss: 0.1071 - acc: 0.9604 - val_loss: 7.8482 - val_acc: 0.5131
Epoch 2/50
  9/666 [..............................] - ETA: 4:43 - loss: 0.0841 - acc: 0.9722